# Exporting and Deploying with Torschscript

In [3]:
import os
import time
import cv2
import torch
import torchvision
import argparse
import numpy as np
from PIL import Image
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog
from detectron2.utils.visualizer import Visualizer, ColorMode


In [19]:
#TODO: remove this stupidness.
os.chdir("..")

In [21]:
yaml_file = "weights/buildings_poc_cfg.yml"
model_weights = "weights/model_final.pth"

In [23]:
cfg = get_cfg()
cfg.merge_from_file(yaml_file)
cfg.MODEL.WEIGHTS = model_weights
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.2
# cfg.MODEL.ROI_HEADS.NUM_CLASSES = 8
cfg.MODEL.DEVICE = "cpu" 
predictor = DefaultPredictor(cfg)

Skip loading parameter 'proposal_generator.rpn_head.conv.weight' to the model due to incompatible shapes: (256, 256, 3, 3) in the checkpoint but (1024, 1024, 3, 3) in the model! You might want to double check if this is expected.
Skip loading parameter 'proposal_generator.rpn_head.conv.bias' to the model due to incompatible shapes: (256,) in the checkpoint but (1024,) in the model! You might want to double check if this is expected.
Skip loading parameter 'proposal_generator.rpn_head.objectness_logits.weight' to the model due to incompatible shapes: (3, 256, 1, 1) in the checkpoint but (15, 1024, 1, 1) in the model! You might want to double check if this is expected.
Skip loading parameter 'proposal_generator.rpn_head.objectness_logits.bias' to the model due to incompatible shapes: (3,) in the checkpoint but (15,) in the model! You might want to double check if this is expected.
Skip loading parameter 'proposal_generator.rpn_head.anchor_deltas.weight' to the model due to incompatible s

In [ ]:
yaml_file = "weights/buildings_poc_cfg.yml"
model_weights = "weights/model_final.pth"
./build/torchscript_mask_rcnn output/model.ts input.jpg tracing

In [25]:
!detectron2/tools/deploy/export_model.py --config-file weights/buildings_poc_cfg.yml \
    --output ./output --export-method tracing --format torchscript \
    MODEL.WEIGHTS weights/model_final.pth \
    MODEL.DEVICE cpu

[10/04 19:49:53 detectron2]: Command line arguments: Namespace(format='torchscript', export_method='tracing', config_file='weights/buildings_poc_cfg.yml', sample_image=None, run_eval=False, output='./output', opts=['MODEL.WEIGHTS', 'weights/model_final.pth', 'MODEL.DEVICE', 'cpu'])
[W init.cpp:836] Warning: Use _jit_set_fusion_strategy, bailout depth is deprecated. Setting to (STATIC, 1) (function operator())
[10/04 19:49:54 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from weights/model_final.pth ...
Skip loading parameter 'proposal_generator.rpn_head.conv.weight' to the model due to incompatible shapes: (256, 256, 3, 3) in the checkpoint but (1024, 1024, 3, 3) in the model! You might want to double check if this is expected.
Skip loading parameter 'proposal_generator.rpn_head.conv.bias' to the model due to incompatible shapes: (256,) in the checkpoint but (1024,) in the model! You might want to double check if this is expected.
Skip loading parameter 'proposal

In [ ]:
def inference(model, inputs):
    # use do_postprocess=False so it returns ROI mask
    inst = model.inference(inputs, do_postprocess=False)[0]
    return [{"instances": inst}]

isinstance(image, np.ndarray) == True
image_tensor = torch.as_tensor(image.astype("float32").transpose(2, 0, 1))
wrapper= TracingAdapter(predictor, inputs=[{"image": image_tensor}], inference_func=inference)